<img src="https://i.imgur.com/gb6B4ig.png" width="400" alt="Weights & Biases" />

# What is W&B?

Weights & Biases (W&B) is a set of machine learning tools that helps you build better models faster.

I'll use this Knowledge-based competition ([Plant Pathology 2021 - FGVC8](https://www.kaggle.com/c/plant-pathology-2021-fgvc8)) to demonstrate some of its features: **Dashboard** (experiment tracking) and **Artifacts** (dataset and model versioning). 

## Why is W&B useful?

**Kaggle competitions require fast-paced model development and evaluation**. There are a lot of components: exploring the training data, training different models, combining trained models in different combinations (ensembling), and so on.

> ⏳ Lots of components = lots of places to go wrong = lots of time spent debugging 

You might miss important details and have to retrain your model, or you might train on the wrong data (information leakage). Or, you might use the wrong model for generating submission. 

This is where W&B comes in:
* **Dashboard** (experiment tracking): Log and visualize experiments in real time = Keep data and results in one convenient place. Consider this as a repository of experiments. 
* **Artifacts** (dataset + model versioning): Store and version datasets, models, and results = Know exactly what data a model is being trained on. 

# 🖥 Dashboard (experiment tracking)

Use the Dashboard as a central place to organize and visualize results from your machine learning models.

* Track metrics
* Visualize results
* Train anywhere
* Stay organized

> 🙌 **Save everything in one place and never lose your progress again** 🙌

## Track metrics

Track model performance in real time and identify problem areas immediately.

## Visualize results

W&B supports a large variety of media types - visualize graphs, images, videos, audio, 3D objects, and [more](https://docs.wandb.ai/guides/track/log#logging-objects).

Plus, the dashboard is **interactive** - hover for more options and information.

![Imgur](https://i.imgur.com/xW4cOSx.gif)

## Train anywhere

The W&B dashboard is **centralized** -  whether you're training on a local machine, lab cluster, or spot instances in the cloud, all of your results get logged to a single place.

![centralized dashboard](https://i.imgur.com/BGgfZj3.png)

## Stay organized

W&B logs data into powerful, querably tables that you can search, filter, sort, and group. This makes it easy to compare thousands of different models and find the best performing model for different tasks. 

![image.png](https://i.imgur.com/qPlykIn.png)

# 🗂 Artifacts (dataset versioning)

Use W&B Artifacts to track and version your datasets, models, dependencies, and results across machine learning pipelines.

Think of an Artifact as a versioned folder of data. Once you've saved something as an Artifact, all modifications are automatically logged, giving you a complete history of changes.

> 🙌 **Keep track of which model was trained on which data** 🙌


### Manage your pipeline / Track data lineage

You get a bird eye view on your entire machine learning workflow. Collaborating with your teammates is easier with such a holistic view of your pipeline.

![img](https://i.imgur.com/dhntZxK.png)

# ⚡️ TL;DR

Weights & Biases helps you **build better models faster**.

* Visualize results in real time in an interactive, centralized dashboard
* Identify how changing data affects the resulting model

Plus, W&B is fast ([get started with 6 lines of code](https://docs.wandb.ai/quickstart)) and flexible ([integrated with every major ML framework](https://docs.wandb.ai/guides/integrations)).

Here's a quick (1:42) intro:

[![thumbnail](https://i.imgur.com/oriwd9L.png)](https://www.youtube.com/watch?v=EIgoKitLUqM&t=92s)


# 🧰 1. Set up W&B

W&B is very easy to set up and integrate.

## 🔵 1a. Install `wandb`

`wandb` (the W&B library), comes baked into your Kaggle kernels!

However, since `wandb` is rapidly improving, I recommend `pip install`-ing the latest version with the `--upgrade` and `-q` flags.

In [ ]:
!pip install --upgrade -q wandb

## 🔵 1b. Import `wandb` and log in

You will need a unique API key to log in to Weights & Biases. 

1. If you don't have a Weights & Biases account, you can go to https://wandb.ai/site and create a FREE account.
2. Access your API key: https://wandb.ai/authorize.

There are two ways you can login using a Kaggle kernel:

1. Run a cell with `wandb.login()`. It will ask for the API key, which you can copy + paste in.
2. You can also use Kaggle secrets to store your API key and use the code snippet below to login. Check out this [discussion post](https://www.kaggle.com/product-feedback/114053) to learn more about Kaggle secrets. 

```
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()

# I have saved my API token with "wandb_api" as Label. 
# If you use some other Label make sure to change the same below. 
wandb_api = user_secrets.get_secret("wandb_api") 

wandb.login(key=wandb_api)
```
More on W&B login [here](https://docs.wandb.ai/ref/cli/wandb-login).

In [ ]:
import wandb
from wandb.keras import WandbCallback

wandb.login()

> 📌 `WandbCallback` is a lightweight Keras callback that I'll use later on.

In [ ]:
# 1. Import other dependencies

import tensorflow as tf
print(tf.__version__)
from tensorflow.keras import layers
from tensorflow.keras import models
import tensorflow_addons as tfa

import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split

In [ ]:
# 2. Set the random seeds

def seed_everything():
    os.environ['TF_CUDNN_DETERMINISTIC'] = '1' 
    np.random.seed(hash("improves reproducibility") % 2**32 - 1)
    tf.random.set_seed(hash("by removing stochasticity") % 2**32 - 1)
    
seed_everything()

In [ ]:
# 3. Set hyperparameters

TRAIN_PATH = '../input/resized-plant2021/img_sz_256/'
AUTOTUNE = tf.data.experimental.AUTOTUNE

CONFIG = dict (
    num_labels = 6,
    train_val_split = 0.2,
    img_width = 224,
    img_height = 224,
    batch_size = 64,
    epochs = 10,
    learning_rate = 0.001,
    architecture = "CNN",
    infra = "Kaggle",
    competition = 'plant-pathology',
    _wandb_kernel = 'ayut'
)

> 📌: Store your hyperparameters as a dictionary, because you can later directly log this config dict to W&B.

# 🔨 2. Build Input Pipeline

After importing `wandb` and other dependencies, you can set up an input pipeline as normal. The Plant Pathology 2021 competition is about multi-label classification. 

> The main objective of the competition is to develop machine learning-based models to accurately classify a given leaf image from the test dataset to a particular disease category, and to identify an individual disease from multiple disease symptoms on a single leaf image.

To build the input pipeline I have used `tf.data` API. 

In [ ]:
# 4. Build input pipeline

# Encode competition-provided labels 
label_to_id = {
    'healthy': 0,
    'scab': 1,
    'frog_eye_leaf_spot': 2,
    'rust': 3,
    'complex': 4,
    'powdery_mildew': 5
}

id_to_label = {value:key for key, value in label_to_id.items()} 

# Helper fu
def make_path(row):
    return TRAIN_PATH+row.image

def parse_labels(row):
    label_list = row.labels.split()
    labels = []
    for label in label_list:
        labels.append(str(label_to_id[label]))
    
    return ' '.join(labels)

# Read train.csv file
df = pd.read_csv('../input/plant-pathology-2021-fgvc8/train.csv')
# Get absolute path
df['image'] = df.apply(lambda row: make_path(row), axis=1)
# Parse labels
df['labels'] = df.apply(lambda row: parse_labels(row), axis=1)

# Look at the dataframe
df.head()

In [ ]:
# 5. Training and validation split

train_df, valid_df = train_test_split(df, test_size=CONFIG['train_val_split'])
print(f'Number of train images: {len(train_df)} and validation images: {len(valid_df)}')

In [ ]:
# 6. Helper functions for input pipeline

@tf.function
def decode_image(image):
    # Convert the compressed string to a 3D uint8 tensor
    image = tf.image.decode_jpeg(image, channels=3)
    
    # Normalize image
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    
    # Resize the image to the desired size
    return image

@tf.function
def load_image(df_dict):
    # Load image
    image = tf.io.read_file(df_dict['image'])
    image = decode_image(image)
    
    # Resize image
    image = tf.image.resize(image, (CONFIG['img_height'], CONFIG['img_width']))
    
    # Parse label
    label = tf.strings.split(df_dict['labels'], sep='')
    label = tf.strings.to_number(label, out_type=tf.int32)
    label = tf.reduce_sum(tf.one_hot(indices=label, depth=CONFIG['num_labels']), axis=0)
    
    return image, label

In [ ]:
# 7. Build data loaders

AUTOTUNE = tf.data.AUTOTUNE

trainloader = tf.data.Dataset.from_tensor_slices(dict(train_df))
validloader = tf.data.Dataset.from_tensor_slices(dict(valid_df))

trainloader = (
    trainloader
    .shuffle(1024)
    .map(load_image, num_parallel_calls=AUTOTUNE)
    .batch(CONFIG['batch_size'])
    .prefetch(AUTOTUNE)
)

validloader = (
    validloader
    .map(load_image, num_parallel_calls=AUTOTUNE)
    .batch(CONFIG['batch_size'])
    .prefetch(AUTOTUNE)
)

In [ ]:
# Data loader sanity check

def show_batch(image_batch, label_batch):
    plt.figure(figsize=(20,20))
    for n in range(25):
        ax = plt.subplot(5,5,n+1)
        plt.imshow(image_batch[n])
        plt.title(' '.join([id_to_label[i] for i, label in enumerate(label_batch[n].numpy()) if label==1.]))
        plt.axis('off')

image_batch, label_batch = next(iter(trainloader))
show_batch(image_batch, label_batch)

# 🔨 3. Set up your model

Build your model definition next. Since it's a multi-label classification task the output activation is `sigmoid`. 

In [ ]:
# 8. Define model: EfficientNetB0 trained on ImageNet as backbone

def get_model():
    base_model = tf.keras.applications.EfficientNetB0(include_top=False, weights='imagenet')
    base_model.trainabe = True

    inputs = layers.Input((CONFIG['img_height'], CONFIG['img_width'], 3))
    x = base_model(inputs, training=True)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(len(label_to_id), activation='sigmoid')(x)
    
    return models.Model(inputs, outputs)

# Model sanity check
tf.keras.backend.clear_session()
model = get_model()
model.summary()

# 🚄 3. Train and evaluate with W&B

In this section I'll use:

* [`wandb.init()`](https://docs.wandb.ai/guides/track/launch): Initialize a new run.

* [`wandb.finish()`](https://docs.wandb.ai/ref/python/finish): Finish and close a run.

* [`wandb.config`](https://docs.wandb.ai/guides/track/config): An object that stores hyperparameters and settings related to a run.


A run (or [wandb.Run object](https://docs.wandb.ai/ref/python/run)) is a W&B unit of computation, typically an ML experiment.

In [ ]:
# Initialize model
tf.keras.backend.clear_session()
model = get_model()

# Compile model
optimizer = tf.keras.optimizers.Adam(learning_rate=CONFIG['learning_rate'])
model.compile(optimizer, 
              loss=tfa.losses.SigmoidFocalCrossEntropy(), 
              metrics=[tf.keras.metrics.AUC(multi_label=True), tfa.metrics.F1Score(num_classes=6, average='micro')])

## 🔵 3. Use `wandb.init()` to initialize a new W&B run.

In an ML training pipeline, you could add `wandb.init()` to the beginning of your training script as well as your evaluation script, and each piece would be tracked as a run in W&B.

Take a note of these arguments. 

* `entity`: An entity is a username or team name where you're sending runs. 
* `project`: The name of the project where you're sending the new run. If the project is not specified, the run is put in an "Uncategorized" project.
* `config`: This sets wandb.config, a dictionary-like object for saving inputs to your job, like hyperparameters for a model or settings for a data preprocessing job. 
* `group`: Specify a group to organize individual runs into a larger experiment.This is a super handy feature. For example, you can create group for different model architecture names. 
* `job_type`: Specify the type of run, which is useful when you're grouping runs together into larger experiments using group. Typical job types are "train", "evaluate", etc. 

In [ ]:
# Update CONFIG dict with the name of the model.
CONFIG['model_name'] = 'efficientnetb0'
print('Training configuration: ', CONFIG)

# Initialize W&B run
run = wandb.init(project='plant-pathology', 
                 config=CONFIG,
                 group='EfficientNet', 
                 job_type='train')

I used these `wandb.init()` arguments:

* `project`: This argument specifies the name of the W&B project where the run gets sent to. Here, I'm creating a new project called `'plant-pathology'` and sending the run there at the same time.

* `config`: This argument sets `wandb.config`, a dictionary-like object that stores hyperparameters, input settings, and other independent variables.

    As a reminder, here's what's currently in `CONFIG`:

```python
CONFIG = dict (
    num_labels = 6,
    train_val_split = 0.2,
    img_width = 224,
    img_height = 224,
    batch_size = 64,
    epochs = 10,
    learning_rate = 0.001,
    architecture = "CNN",
    infra = "Kaggle",
    model_name = "efficientnetb0"
)
```

* `group`: This argument specifies a value to group individual runs by. Later on, I'll have a run that groups by `'EfficientNet'`, so that I can compare runs from the different architectures more easily.
* `job_type`: This argument specifies a run type, for example `'train'`, or `'evaluate'`. Setting a run type makes it easier to later filter and group runs, for example if you want to compare multiple `'train'` runs.

## 🔵 3b. Update `wandb.config`

Saving your training configuration is useful for analyzing your experiments and reproducing your work later. With W&B, you can also group by config values, meaning that you can compare the settings of different runs and see how they affect the output.

There are multiple ways to set up `wandb.config`:

* Set `wandb.config` with the `wandb.init(config)` argument, as above in (3a). 
* Set `wandb.config` directly.
* See more setup options in this [Colab](https://colab.research.google.com/github/wandb/examples/blob/master/colabs/wandb-log/Configs_in_W%26B.ipynb#scrollTo=xFf3zjBSixC1).

In [ ]:
# Add "type" and "kaggle_competition" to `wandb.config` directly
wandb.config.type = 'baseline'
wandb.config.kaggle_competition = 'Plant Pathology 2021 - FGVC8'

Here's a gif showing how W&B logs and displays a run's training configuration:

![img](https://i.imgur.com/WxLjIBx.gif)

Now I'll use `WandbCallback()`, the lightweight Keras integration that I mentioned earlier.

This callback automatically saves all the metrics and the loss values tracked in `model.fit()`. Check out our [Keras integration docs](https://docs.wandb.ai/guides/integrations/keras) to learn more.

Weights and Biases comes with a light weight integration for Keras. We will be using W&B Keras integration (`WandbCallback()`) to automatically save all the metrics and the loss values tracked in `model.fit()`. Check out the [docs to learn more about this integration](https://docs.wandb.ai/guides/integrations/keras). 

In [ ]:
earlystopper = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=3, verbose=0, mode='min',
    restore_best_weights=True
)

# Train
model.fit(trainloader, 
          epochs=CONFIG['epochs'],
          validation_data=validloader,
          callbacks=[WandbCallback(),
                     earlystopper])

# Close W&B run
run.finish()

> 📌 Pro tip: Head over to the W&B dashboard my clicking on the link generated above. 

> 📌 Pro tip: If you want to silence W&B related logs use this code snippet `os.environ[WANDB_SILENT] = "true"` after `import os`. Check out this [Stackoverflow answer](https://stackoverflow.com/a/65997094/8663152) for more details. 

> 📌 Pro tip: Use `run.finish()` to close the initialized W&B run after a `job_type` is finished. 

Each run gets its own Run Page, which has tabs that contain more information about the run.

Here's a gif showing the [Charts Tab](https://docs.wandb.ai/ref/app/pages/run-page#charts-tab), [System Tab](https://docs.wandb.ai/ref/app/pages/run-page#system-tab), and [Model Tab](https://docs.wandb.ai/ref/app/pages/run-page#model-tab) from the run above:

![img](https://i.imgur.com/Eq8X9RN.gif)

You can click through this Run Page [here](https://wandb.ai/ayush-thakur/plant-pathology/runs/5yq0kz7t).

### Filtering and Grouping

You can use filter and group feature on W&B dashboard to either hide crashed run, group together multiple runs under one experiment, group according to job_type, select runs that satisfy a condition, etc. You can learn more about Group feature [here](https://docs.wandb.ai/guides/track/advanced/grouping).

![img](https://i.imgur.com/BeYKbfS.gif)

## 🔵 3c. Log evaluation score with `wandb.log()`

`WandbCallback()` can be used for `model.fit()`, but not for `model.evaluate()`. If you want to log metrics from evaluation, you should call `wandb.log()`.

* [`wandb.log()`](https://docs.wandb.ai/guides/track/log): Log a dict of scalars (metrics like accuracy and loss) and any other type of [`wandb` object](https://docs.wandb.ai/ref/python/data-types).

In [ ]:
# Initialize a run
run = wandb.init(project='plant-pathology', 
                 config=CONFIG,
                 group='EfficientNet', 
                 job_type='evaluate') # Note the job_type

# Update `wandb.config`
wandb.config.type = 'baseline'
wandb.config.kaggle_competition = 'Plant Pathology 2021 - FGVC8'

# Evaluate model
loss, auc, f1_score = model.evaluate(validloader)

# Log scores using wandb.log()
wandb.log({'val_AUC': auc, 
           'val_F1_score': f1_score})

# Finish the run
run.finish()

![img](https://i.imgur.com/OBL9F1i.gif)

> 📌 Pro tip: Note that the bar chart apprears if there are more than one value for a key. 

# 💾 4. Create a W&B Artifact

W&B's Dashboard lets you log the model training **process**, things like output logs, code versions, configuration, hyperparameters, and metrics. W&B Artifacts lets you log the **data** that goes in (like a dataset) and out (like trained model weights) of these processes.

In other words, Artifacts are a way to save your datasets and models. You can use [this Colab](https://colab.research.google.com/github/wandb/examples/blob/master/colabs/wandb-artifacts/Pipeline_Versioning_with_W%26B_Artifacts.ipynb) to learn more about Artifacts.

In this section I'll show you how to create a model Artifact. I also created some dataset Artifacts for this competition, and you can check them all out [here](https://wandb.ai/ayush-thakur/plant-pathology/artifacts).

## 🔵 4a. Save your hard work with `wandb.log_artifact()`

Within a run, there are three steps for creating and saving a model Artifact.

1. Create an empty Artifact with `wandb.Artifact()`.
2. Add your model file to the Artifact with `wandb.add_file()`.
3. Call `wandb.log_artifact()` to save the Artifact.

In [ ]:
# Save model
model.save('efficientnetb0-baseline.h5')

# Initialize a new W&B run
run = wandb.init(project='plant-pathology', 
                 config=CONFIG,
                 group='EfficientNet', 
                 job_type='save') # Note the job_type

# Update `wandb.config`
wandb.config.type = 'baseline'
wandb.config.kaggle_competition = 'Plant Pathology 2021 - FGVC8'

# Save model as Model Artifact
artifact = wandb.Artifact(name='efficientnet-b0', type='model')
artifact.add_file('efficientnetb0-baseline.h5')
run.log_artifact(artifact)

# Finish W&B run
run.finish()

Here's a gif that shows how an Artifact appears within a Run Page.

![img](https://i.imgur.com/HjHDoSx.gif)

At the end of the gif, there's a graph that shows the lineage: the `deep-thunder-8` "save" run produced the `efficientnetb0-v0` model.

![img](https://i.imgur.com/59IFYoT.png)

# ❄️ Resources

I hope you find this kernel useful and I encourage you to try out Weights & Biases. Here are some relevant links that you might want to check out:

* Check out the [official documentation](https://docs.wandb.ai/) to learn more about the best practices and advanced features. 

* Check out the [examples GitHub repository](https://github.com/wandb/examples) for curated and minimal examples. This can be a good starting point. 

* [Fully Connected](https://wandb.ai/fully-connected) is a home for curated tutorials, free-form dicussions, paper summaries, industry expert advices and more. 

Here are some other Kaggle kernels instrumented with Weights & Biases that you might find useful. 

* [EfficientNet+Mixup+K-Fold using TF and wandb](https://www.kaggle.com/ayuraj/efficientnet-mixup-k-fold-using-tf-and-wandb)

* [HPA: Segmentation Mask Visualization with W&B](https://www.kaggle.com/ayuraj/hpa-segmentation-mask-visualization-with-w-b)

* [HPA: Multi-Label Classification with TF and W&B](https://www.kaggle.com/ayuraj/hpa-multi-label-classification-with-tf-and-w-b)

* [🐦BirdCLEF: Quick EDA with W&B](https://www.kaggle.com/ayuraj/birdclef-quick-eda-with-w-b)